# 라이브러리 설치

In [17]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 42.6 MB/s eta 0:00:00


# 라이브러리 선언

In [1]:
import pandas as pd
from sqlalchemy import create_engine, inspect

# 구글드라이브 연동

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

In [5]:
%cd /content/gdrive/MyDrive/Colab Notebooks/analysis_edu/05. 데이터분석 및 데이터 전처리

/content/gdrive/MyDrive/Colab Notebooks/analysis_edu/05. 데이터분석 및 데이터 전처리


# 파일 불러오기

In [11]:
googleUrl = "../dataset/customerdata.csv"
csDataGitUrl = "https://raw.githubusercontent.com/hyokwan/python-lecture/master/dataset/customerdata.csv"

In [23]:
customerData = pd.read_csv(csDataGitUrl)
customerData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   CUSTID       150 non-null    object 
 1   AVGPRICE     150 non-null    float64
 2   EMI          150 non-null    int64  
 3   DEVICECOUNT  150 non-null    int64  
 4   PRODUCTAGE   150 non-null    float64
 5   CUSTTYPE     150 non-null    object 
dtypes: float64(2), int64(2), object(2)
memory usage: 7.2+ KB


In [31]:
# 테이블정의서 저장
typeDf  = pd.DataFrame( customerData.dtypes )
typeDf.to_csv("./테이블정의서.csv", index=True)

### 1. 원하는 행 조회하기 (기본조건)

In [47]:
emiCond = 2
avgPriceCond = 3500

In [82]:
customerData.loc[  (customerData.EMI > emiCond) &
                   (customerData.AVGPRICE >= avgPriceCond) ].shape

(11, 6)

In [83]:
# ★ 조건검증 꼭 해보기
customerData.loc[  (customerData.EMI <= emiCond) |
                   (customerData.AVGPRICE < avgPriceCond) ].shape

(139, 6)

### 1. 원하는 행 조회하기 (in not in 조건)

In [85]:
emiMultiCond = [2,3]

In [208]:
customerData.loc[ ~ customerData.EMI.isin( emiMultiCond )  ].head(2)

,CUSTID,AVGPRICE,EMI,DEVICECOUNT,PRODUCTAGE,CUSTTYPE,EMI_NEW
0,A13566,4273.9,3,6,1.67918084803969,Big-Screen-lover,3
1,A14219,3642.441950493661,2,4,2.68202302093618,Sleeping-dog,2


### 1. 원하는 행 조회하기 (컬럼 값 세부 조건)

In [ ]:
 customerData.loc [customerData.CUSTID.astype(str).str[0] == "A"]

### 컬럼명 조작방법

In [68]:
# 데이터프레임 컬럼 정보를 시연!
customerData.columns

Index(['CUSTID', 'AVGPRICE', 'EMI', 'DEVICECOUNT', 'PRODUCTAGE', 'CUSTTYPE'], dtype='object')

In [75]:
# 데이터프레임 컬럼정보 수정 * 문자열함수를 사용하려면 .str을 df에서는 써야한다!
customerData.columns = customerData.columns.str.upper()

In [76]:
## 전체 데이터 프레임의 크기 확인용
customerData.shape

(150, 6)

### 2. 데이터 타입변환 (astype)

In [210]:
!pip show sqlalchemy

Name: SQLAlchemy
Version: 2.0.31
Summary: Database Abstraction Library
Home-page: https://www.sqlalchemy.org
Author: Mike Bayer
Author-email: mike_mp@zzzcomputing.com
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: greenlet, typing-extensions
Required-by: bigframes, ipython-sql


In [99]:
customerData.EMI = customerData.EMI.astype(str)

<ipython-input-99-18340af54f26>:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  customerData.EMI_NEW = customerData.EMI.astype(str)


In [107]:
columnNameList = list( customerData.columns )

In [110]:
for i in range(0, len(columnNameList)):
    customerData[ columnNameList[i] ] = customerData[ columnNameList[i] ].astype(str)

### 3. 인덱스 초기화

In [162]:
targetColumns = ["AVGPRICE","EMI"]
# 컬럼 이름으로 조회
customerDataIndex = customerData.loc[ customerData.AVGPRICE.astype(float) < 3000 ,
                  targetColumns ].reset_index(drop=True)

### 핵심 데이터 조회를 통해 값을 받아 처리

In [194]:
paramDf = pd.read_csv("./paramData.txt")
ipName = "HOST"
portName = "PORT"
userIdName = "USERID"
userPassName = "USERPASS"
dbName = "DBNAME"
targetColum = ["PARAM_VALUE"]
dbIp = paramDf.loc[ paramDf.PARAM_NAME == ipName,targetColum].values[0][0]
dbPort = paramDf.loc[ paramDf.PARAM_NAME == portName,targetColum].values[0][0]
dbUserName = paramDf.loc[ paramDf.PARAM_NAME == userIdName,targetColum].values[0][0]
dbUserPass = paramDf.loc[ paramDf.PARAM_NAME == userPassName,targetColum].values[0][0]
dbName = paramDf.loc[ paramDf.PARAM_NAME == dbName,targetColum].values[0][0]

# DB 데이터 저장 (MySQL)

In [206]:
# MySQL 데이터베이스 연결 정보 설정 (클라우드)
host = dbIp
port = dbPort
user = dbUserName
password = dbUserPass
database = dbName

# SQLAlchemy 엔진 생성
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}')

# 데이터프레임을 MySQL 데이터베이스의 테이블에 저장
table_name = 'paramdf_0731_final'
paramDf.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

5